In [14]:
import os
root = os.getcwd().rsplit('/',1)[0]

vocab_path = '{0:s}/etc/vocab.ttl'.format(root)
dict_path = '{0:s}/etc/dict.ttl'.format(root)

import sys
sys.path.append('/global/homes/s/sleak/.local/cori/3.6-anaconda-4.4/lib/python3.6/site-packages')
import rdflib
graph = rdflib.ConjunctiveGraph().parse(vocab_path, format='turtle')
# now add the data dictionary:
graph.parse(dict_path, format='turtle')
print ("graph has {0} statements".format(len(graph)))
print (os.getcwd())

graph has 97 statements
/global/u1/s/sleak/Monitoring/Resilience/LogSet/prototypes


In [2]:
# get a list of filename patterns we know what to do with:
query = '''SELECT ?series ?fmt ?fmtinfo
WHERE {
?series a logset:LogSeries .
?series logset:logFormat ?fmt .
?series logset:logFormatInfo ?fmtinfo .
}
'''

#query = '''SELECT ?series ?filepattern ?seriespattern
#WHERE {
#?series a :LogSeries .
#?series :filePattern ?fp .
#?fp rdf:value ?filepattern .
#?series :seriesPattern ?seriespattern .
#}
#'''
class F1:
    name = 'timeStampedLogFile'
    init_args = {}       # 'console_logfile': {ts_word=0, part_word=1}
    patterns = {} # 'console': 'console_logfile', 'console-<dateYYYYMMDD>': 'console_logfile'
    def __init__(self, name, root_path=os.getcwd(), rel_path='', ts_word='0', part_word='1'):
        self.name = name         # filename (only - no path components)
        self.root_path = root_path # where to start looking to open this file
        self.rel_path = rel_path   # path from rootpath o file
        self.ts_word = int(ts_word) # which "word" in each line has the timestamp?
        self.part_word = int(part_word)
        print ("created a {0} with tsword {1:d} and partword {2:d}".format(self.name, self.ts_word, self.part_word))

        # make another class basically the same (for testing)
class F2(F1):
    name = 'unstructuredFile'

        
#a = F1('dummy', ts_word=0, part_word=1)
#F1.init_args['dummy_logfile'] = {'ts_word':3, 'part_word':0}
#b = F2('dummy2', **{'ts_word':3, 'part_word':0})
#b = F1('dummy3', **(F1.init_args['dummy_logfile']))

mapping = {c.name: c for c in (F1, F2)}
print(mapping)

formattypes = {}  # fmt: 
for row in graph.query(query):
    label = row[0].rpartition('#')[2]
    fmt = row[1].rpartition('#')[2]
    info = row[2].rpartition('#')[2]
    print('{0}  {1}  {2}'.format(label, fmt, info))
    
    # build it into a dictionary of filepatterns 
    # I ultimately want a filepattern: what_to_instantiate mapping
    k,sep,v = info.partition('=')
    if k == 'filepattern':
        mapping[fmt].patterns[v] = label
    else:
        # hmm, need to convert it from a string to suitable type
        # maybe accept strings and convert within the class, then the class looks after the how
        mapping[fmt].init_args[k] = v

for fmt in mapping:
    print (mapping[fmt].init_args)
    print (mapping[fmt].patterns)
    
# ok .. need to convert patterns to regexs
# then walk fs for files whose name matches the pattern


{'timeStampedLogFile': <class '__main__.F1'>, 'unstructuredFile': <class '__main__.F2'>}
messages_logfile  timeStampedLogFile  part_word=2
messages_logfile  timeStampedLogFile  filepattern=messages
messages_logfile  timeStampedLogFile  ts_word=1
messages_logfile  timeStampedLogFile  filepattern=messages-<dateYYYYMMDD>
console_logfile  timeStampedLogFile  filepattern=console-<dateYYYYMMDD>
console_logfile  timeStampedLogFile  part_word=1
console_logfile  timeStampedLogFile  ts_word=0
console_logfile  timeStampedLogFile  filepattern=console
consumer_logfile  unstructuredFile  filepattern=consumer-<dateYYYYMMDD>
consumer_logfile  unstructuredFile  filepattern=consumer
{'part_word': '1', 'ts_word': '0'}
{'messages': 'messages_logfile', 'messages-<dateYYYYMMDD>': 'messages_logfile', 'console-<dateYYYYMMDD>': 'console_logfile', 'console': 'console_logfile', 'consumer-<dateYYYYMMDD>': 'consumer_logfile', 'consumer': 'consumer_logfile'}
{'part_word': '1', 'ts_word': '0'}
{'messages': 'messages

In [3]:
target_path = '{0:s}/examples/p0-20170906t151820'.format(root)
baselen = len(target_path)+1
for d,f in [(base[baselen:],file) for base, dir, files in os.walk(target_path) for file in files]:
    print((d,f))


('', 'messages-20170907')
('', 'dumpd-20170906')
('', 'console-20170907')
('', 'consumer-20170906')
('', 'console-20170906')
('', 'consumer-20170907')
('', 'messages-20170906')
('', 'hwerrlog.p0-20170906t151820')
('', 'craylog.p0-20170906t151820')
('nersc.aries_lcb', 'cori_aries_lcb_status.20170907213001.xz')
('nersc.aries_lcb', 'cori_aries_lcb_status.20170907053002.xz')
('nersc.aries_lcb', 'cori_aries_lcb_status.20170907232501.xz')
('nersc.aries_lcb', 'cori_aries_lcb_status.20170907052501.xz')
('nersc.aries_lcb', 'cori_aries_lcb_status.20170906190001.xz')
('nersc.aries_lcb', 'cori_aries_lcb_status.20170906185001.xz')
('nersc.aries_lcb', 'cori_aries_lcb_status.20170907094501.xz')
('nersc.aries_lcb', 'cori_aries_lcb_status.20170906181001.xz')
('nersc.aries_lcb', 'cori_aries_lcb_status.20170907074502.xz')
('nersc.aries_lcb', 'cori_aries_lcb_status.20170907072501.xz')
('nersc.aries_lcb', 'cori_aries_lcb_status.20170906214501.xz')
('nersc.aries_lcb', 'cori_aries_lcb_status.20170906235501.x

In [4]:
line = "2017-09-06T15:20:21.827169-07:00 corismw1 craylog: message type: console, connected to port: 5150"

import dateutil.parser
dateutil.parser.parse(line.split()[0])

line2 = "<30>1 2017-09-06T15:26:39.977558-07:00 c0-0c0s0n1 systemd-modules-load 5894 p0-20170906t151820 -  Inserted module 'mlx4_en'"
dateutil.parser.parse(line2.split()[1])
#class TimeStampedLogFile:

datetime.datetime(2017, 9, 6, 15, 26, 39, 977558, tzinfo=tzoffset(None, -25200))

In [5]:
blocksz = 10000 # arbirtrary, large-but-not-too-large, amount of data to read while searching file
testfile = os.path.join(target_path, 'console-20170907')
sz = os.path.getsize(testfile)
with open(testfile, 'rb') as f:
    f.seek(sz//2)
    junkline= f.readline()
    pos = f.tell()
junkline
with open(testfile, 'rb') as f:
    bs = min(blocksz, sz)    
    lines = []
    while bs <= sz:
        f.seek(-bs, 2)
        lines = f.readlines() # read to end of file
        if len(lines) > 1:
            break
        print ("increasing bs")
        bs *= 2
    else:
        raise Exception("can't find last entry in {0:s}".format(testfile))
    print (lines[-1])
dateutil.parser.parse(lines[-1].split()[0])

b'2017-09-07T00:01:31.068610-07:00 c7-5c2s10n1 nersc-nodeprolog|6680771|nid13033|2017-09-07T00:01:25\n'


datetime.datetime(2017, 9, 7, 0, 1, 31, 68610, tzinfo=tzoffset(None, -25200))

In [6]:
from TimeStampedLogFile import TimeStampedLogFile
sample = TimeStampedLogFile('console-20170906',rootpath=target_path)
print(str(sample.timespan()))
sample = TimeStampedLogFile('messages-20170906',rootpath=target_path, tsword=1)
print(str(sample.timespan()))

(1504736421.827169, 1504736562.809617)
(1504736799.977558, 1504736957.665618)


In [7]:
import sys
sys.version_info

sys.version_info(major=3, minor=6, micro=1, releaselevel='final', serial=0)

In [12]:
def dummy(*urls, local_only=True):
    for url in urls:
        print(url)
    if local_only:
        print("local only")
    else:
        print ("spider")
        
dummy("abc", "def")
dummy("123", local_only=False)


abc
def
local only
123
spider
